In [0]:
-- 1. Business_DIM_CHI  (SCD1 business dimension for reporting)
CREATE OR REFRESH LIVE TABLE Business_DIM_CHI AS
SELECT DISTINCT
  business_key,
  license_number,
  dba_name,
  aka_name
FROM workspace.food_inspection_project.silver_business_chi;


In [0]:
-- 2. Facility_DIM_CHI
CREATE OR REFRESH LIVE TABLE Facility_DIM_CHI AS
SELECT DISTINCT
  facility_key,
  facility_type
FROM workspace.food_inspection_project.silver_facility_chi;


In [0]:
-- 3. Result_DIM_CHI
CREATE OR REFRESH LIVE TABLE Result_DIM_CHI AS
SELECT DISTINCT
  result_key,
  result
FROM workspace.food_inspection_project.silver_result_chi;


In [0]:
-- 4. Risk_DIM_CHI
CREATE OR REFRESH LIVE TABLE Risk_DIM_CHI AS
SELECT DISTINCT
  risk_key,
  risk_level
FROM workspace.food_inspection_project.silver_risk_chi;


In [0]:
-- 5. Location_DIM_CHI
-- (built directly from the exploded Silver inspection table)
CREATE OR REFRESH LIVE TABLE Location_DIM_CHI AS
SELECT DISTINCT
  xxhash64(city, zip, address, state, latitude, longitude) AS location_key,
  city,
  zip        AS zip_code,
  address    AS street_address,
  state,
  latitude,
  longitude
FROM workspace.food_inspection_project.silver_food_inspection;


In [0]:
-- 6. Violation_DIM_CHI
-- NOTE: source column name is violation_key (NOT violation_code)
CREATE OR REFRESH LIVE TABLE Violation_DIM_CHI AS
SELECT DISTINCT
  violation_key,
  violation_description
FROM workspace.food_inspection_project.silver_violation_chi;


In [0]:
-- FACT_Inspection_CHI
-- Grain: one row per (inspection, violation)
-- Includes inspection_key, inspection_score (placeholder), and violation_cnt

CREATE OR REFRESH LIVE TABLE Fact_Inspection_CHI
AS
WITH base AS (
  SELECT
    inspection_id,
    license_no,
    restaurant_name,
    aka_name,
    facility_type,
    address,
    city,
    state,
    zip,
    inspection_date,
    inspection_type,
    results,
    risk,
    violation_code,
    violation_description
  FROM workspace.food_inspection_project.silver_food_inspection
),

joined AS (
  SELECT
    b.business_key,
    f.facility_key,
    l.location_key,
    r.result_key,
    rk.risk_key,
    v.violation_key,

    idim.inspection_key,   -- from inspection dim

    base.*
  FROM base
  LEFT JOIN LIVE.Business_DIM_CHI     b
    ON  base.license_no      = b.license_number
    AND base.restaurant_name = b.dba_name
    AND base.aka_name        = b.aka_name
  LEFT JOIN LIVE.Facility_DIM_CHI     f
    ON base.facility_type = f.facility_type
  LEFT JOIN LIVE.Location_DIM_CHI     l
    ON UPPER(base.city) = UPPER(l.city)
   AND base.zip         = l.zip_code
   AND base.address     = l.street_address
  LEFT JOIN LIVE.Result_DIM_CHI       r
    ON base.results = r.result
  LEFT JOIN LIVE.RISK_DIM_CHI         rk
    ON base.risk = rk.risk_level
  LEFT JOIN LIVE.Violation_DIM_CHI    v
    ON base.violation_code        = v.violation_key
   AND base.violation_description = v.violation_description
  LEFT JOIN LIVE.Inspection_DIM_CHI   idim
    ON base.inspection_id = idim.inspection_id
)

SELECT
  -- Fact key: here equal to inspection_id (you can xxhash64 if you want)
  inspection_id                                        AS inspection_fact_key,

  -- Date key for joining to Date_DIM (if you have it)
  CAST(date_format(inspection_date, 'yyyyMMdd') AS INT) AS date_key,

  -- Foreign keys
  inspection_key,
  business_key,
  facility_key,
  location_key,
  result_key,
  risk_key,
  violation_key,

  -- Degenerate / descriptive fields
  inspection_id,
  inspection_date,
  inspection_type,
  results,
  risk,
  violation_code,
  violation_description,

  -- Metrics:
  1 AS violation_cnt,
  CAST(NULL AS INT) AS inspection_score        -- placeholder; Chicago has no score column
FROM joined;


In [0]:
-- BRIDGE_Inspection_Violation_CHI
-- Many-to-many bridge between inspections and violations

CREATE OR REFRESH LIVE TABLE BRIDGE_Inspection_Violation_CHI AS
SELECT
  inspection_fact_key,
  violation_key,
  COUNT(*) AS violation_cnt
FROM LIVE.Fact_Inspection_CHI
GROUP BY
  inspection_fact_key,
  violation_key;


In [0]:
-- =========================================================
-- DATE_DIM_CHI (Gold Date Dimension)
-- =========================================================
CREATE OR REFRESH LIVE TABLE date_dim_chi AS
SELECT DISTINCT
    /* Surrogate key – YYYYMMDD as integer */
    CAST(date_format(inspection_date, 'yyyyMMdd') AS INT) AS date_key,

    inspection_date                                        AS date_actual,
    year(inspection_date)                                  AS year,
    month(inspection_date)                                 AS month,
    day(inspection_date)                                   AS day,
    date_format(inspection_date, 'EEEE')                   AS weekday_name,
    weekofyear(inspection_date)                            AS week_of_year,
    quarter(inspection_date)                               AS quarter
FROM workspace.food_inspection_project.silver_inspection_chi
WHERE inspection_date IS NOT NULL;


In [0]:
-- Inspection_DIM_CHI: one row per inspection
CREATE OR REFRESH LIVE TABLE Inspection_DIM_CHI AS
SELECT DISTINCT
  xxhash64(inspection_id) AS inspection_key,
  inspection_id,
  inspection_date,
  inspection_type
FROM workspace.food_inspection_project.silver_inspection_chi;
